# 图片离线放大

70–100px 的原图直接训 Pose 很难稳，先离线放大是关键（不用改标签）。

In [8]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
from pathlib import Path
import cv2, shutil


TARGET_MIN = 640      # 短边放大到这个尺寸（可改 320/384/512）

def upscale(img):
    h, w = img.shape[:2]
    if min(h, w) >= TARGET_MIN:
        return img
    s = TARGET_MIN / min(h, w)
    nh, nw = int(round(h*s)), int(round(w*s))
    # Lanczos 放大
    up = cv2.resize(img, (nw, nh), interpolation=cv2.INTER_LANCZOS4)
    # 轻微锐化（unsharp mask）
    blur = cv2.GaussianBlur(up, (0,0), 1.0)
    sharp = cv2.addWeighted(up, 1.15, blur, -0.15, 0)
    return sharp

def process_split(split, SRC, LAB, DST,LAB_DST):
    for p in (SRC/split).rglob("*"):
        if p.suffix.lower() not in {".jpg",".jpeg",".png"}: 
            continue
        rel = p.relative_to(SRC)
        img = cv2.imread(str(p))
        if img is None: 
            continue
        out_img = upscale(img)
        out_path = (DST/rel).with_suffix(".jpg")
        out_path.parent.mkdir(parents=True, exist_ok=True)
        cv2.imwrite(str(out_path), out_img, [int(cv2.IMWRITE_JPEG_QUALITY), 95])

        # 复制同名 label
        lab_src = (LAB/split/rel).with_suffix(".txt")
        lab_dst = (LAB_DST/split/rel).with_suffix(".txt")
        lab_dst.parent.mkdir(parents=True, exist_ok=True)
        if lab_src.exists():
            shutil.copy2(lab_src, lab_dst)
        else:
            # 没有标签就跳过（你也可以写个0字节空txt）
            pass


# 分割原始数据为 --> train-val-test

In [9]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import shutil
import random
from pathlib import Path
from typing import List, Tuple

# ================= 配置 =================
# 源数据：假设图片与同名 .txt 标签位于同一目录或其子目录中
SOURCE_DIR = Path("cropped_objects")

# 图片/标签扩展名
IMG_EXTS = {".jpg", ".jpeg", ".png"}
LABEL_EXT = ".txt"

# 其他选项
SEED = 42
USE_SYMLINK = False        # True=软链接，False=复制
FORCE_CLEAN = False        # True=若目标已存在则清空
ALLOW_NEGATIVE = False     # True=允许没有任何目标的“空txt”样本；False=跳过无label样本

# 可选：自动写出一个最小 YAML（方便后续训练）
WRITE_YAML = True
YAML_NAME = "swd_pose.yaml"
KPT_SHAPE = [3, 3]         # 3个关键点，每点(x,y,v)
FLIP_IDX = [0, 2, 1]
SKELETON = [[0, 1], [0, 2]]
# NAMES = ["swd"] 
NAMES = ["swd","p"]            # 只有1类时


# ================ 工具函数 ================
def _list_image_files(root: Path) -> List[Path]:
    return [p for p in root.rglob("*") if p.is_file() and p.suffix.lower() in IMG_EXTS]

def _has_label(img_path: Path) -> bool:
    return img_path.with_suffix(LABEL_EXT).exists()

def _copy_or_link(src: Path, dst: Path, symlink: bool = False):
    dst.parent.mkdir(parents=True, exist_ok=True)
    if symlink:
        if dst.exists() or dst.is_symlink():
            dst.unlink()
        os.symlink(src.resolve(), dst)
    else:
        shutil.copy2(src, dst)

def _ensure_empty_dir(d: Path):
    if d.exists():
        if FORCE_CLEAN:
            shutil.rmtree(d)
            d.mkdir(parents=True, exist_ok=True)
        else:
            # 若不强制清空，要求目录为空；避免误覆盖
            if any(d.rglob("*")):
                raise SystemExit(f"[ABORT] 目标目录已存在且非空：{d}\n"
                                 f" - 修改 DEST_DIR 或设置 FORCE_CLEAN=True 再运行。")
    else:
        d.mkdir(parents=True, exist_ok=True)

def _write_yaml(dest_root: Path):
    yaml_path = dest_root / YAML_NAME
    content = [
        f"path: {dest_root.resolve()}",
        "train: images/train",
        "val: images/val",
        "test: images/test",
        "",
        f"kpt_shape: [{KPT_SHAPE[0]}, {KPT_SHAPE[1]}]",
        f"flip_idx: [{', '.join(map(str, FLIP_IDX))}]",
        "skeleton:"
    ]
    for a, b in SKELETON:
        content.append(f"  - [{a}, {b}]")
    content.append("")
    content.append("names:")
    for i, n in enumerate(NAMES):
        content.append(f"  {i}: {n}")
    yaml_path.write_text("\n".join(content), encoding="utf-8")
    print(f"[OK] 写出 YAML: {yaml_path}")


# ================ 主流程 ================
def main(SPLITS: dict, DEST_DIR: Path):
    assert 0 < SPLITS["train"] < 1 and 0 < SPLITS["val"] < 1 and 0 < SPLITS["test"] < 1, "SPLITS 每项需在 (0,1) 内"
    if abs(sum(SPLITS.values()) - 1.0) > 1e-6:
        print(f"[WARN] 划分比例之和={sum(SPLITS.values()):.6f} ≠ 1.0，将以 test = N - train - val 兜底。")

    random.seed(SEED)

    # 1) 收集样本（图片+同名txt）
    imgs = _list_image_files(SOURCE_DIR)
    pairs: List[Tuple[Path, Path]] = []
    neg_imgs: List[Path] = []  # 没有 label 的图片

    for img in imgs:
        lab = img.with_suffix(LABEL_EXT)
        if lab.exists():
            if lab.stat().st_size == 0 and not ALLOW_NEGATIVE:
                # 空txt当作负样本，默认跳过
                continue
            pairs.append((img, lab))
        else:
            neg_imgs.append(img)

    if not pairs:
        raise SystemExit("[ABORT] 没有找到成对的 图片+同名.txt；请检查 SOURCE_DIR 或标签生成。")

    print(f"[INFO] 收集到样本对：{len(pairs)}（跳过负样本：{len(neg_imgs)}）")

    # 2) 打乱并划分
    random.shuffle(pairs)
    n = len(pairs)
    n_train = int(SPLITS["train"] * n)
    n_val = int(SPLITS["val"] * n)
    n_test = n - n_train - n_val
    assert n_train >= 0 and n_val >= 0 and n_test >= 0

    splits = {
        "train": pairs[:n_train],
        "val": pairs[n_train:n_train + n_val],
        "test": pairs[n_train + n_val:]
    }

    print("[INFO] 划分结果：",
          f"train={len(splits['train'])}, val={len(splits['val'])}, test={len(splits['test'])}")

    # 3) 创建目标结构
    _ensure_empty_dir(DEST_DIR)
    for split in ("train", "val", "test"):
        (DEST_DIR / "images" / split).mkdir(parents=True, exist_ok=True)
        (DEST_DIR / "labels" / split).mkdir(parents=True, exist_ok=True)

    # 4) 复制/链接文件
    for split, items in splits.items():
        for img, lab in items:
            dst_img = DEST_DIR / "images" / split / img.name
            dst_lab = DEST_DIR / "labels" / split / (img.stem + LABEL_EXT)
            _copy_or_link(img, dst_img, symlink=USE_SYMLINK)
            _copy_or_link(lab, dst_lab, symlink=USE_SYMLINK)

    # 5) 统计与 YAML
    for split in ("train", "val", "test"):
        ni = len(list((DEST_DIR / "images" / split).glob("*")))
        nl = len(list((DEST_DIR / "labels" / split).glob("*")))
        print(f"[OK] {split:<5} images={ni}  labels={nl}")

    if WRITE_YAML:
        _write_yaml(DEST_DIR)

    print("✅ Done. 数据已整理为 YOLO-Pose 标准结构。")
    print(f"   训练命令示例：\n"
          f"   yolo mode=checks data={DEST_DIR / YAML_NAME}\n"
          f"   yolo pose train model=yolo11s-pose.pt data={DEST_DIR / YAML_NAME} "
          f"imgsz=1280 epochs=150 batch=16 mosaic=0 fliplr=0.5 degrees=5 scale=0.2")




In [10]:
if __name__ == "__main__":
    # 划分比例：合计≈1.0；最终以 test = N - train - val 兜底
    SPLITS = {"train": 0.7, "val": 0.2, "test": 0.1}
    # 目标数据根目录（将生成 images/{split} 与 labels/{split}）
    DEST_DIR = Path(f"datasets/swd_pose_split_{SPLITS['train']}_{SPLITS['val']}_{SPLITS['test']}")
    main(SPLITS, DEST_DIR)
    # for split in ["train","val","test"]:
    #     SRC = DEST_DIR / "images"  # 原 images 根
    #     LAB = DEST_DIR / "labels"   # 原 labels 根
    #     DST = DEST_DIR / "images"  # 放大后的 images 根
    #     LAB_DST = DEST_DIR / "labels" # labels 同步复制
    #     process_split(split, SRC, LAB, DST, LAB_DST)

    print("✅ Upscale done.")

[INFO] 收集到样本对：887（跳过负样本：0）
[INFO] 划分结果： train=620, val=177, test=90
[OK] train images=620  labels=620
[OK] val   images=177  labels=177
[OK] test  images=90  labels=90
[OK] 写出 YAML: datasets/swd_pose_split_0.7_0.2_0.1/swd_pose.yaml
✅ Done. 数据已整理为 YOLO-Pose 标准结构。
   训练命令示例：
   yolo mode=checks data=datasets/swd_pose_split_0.7_0.2_0.1/swd_pose.yaml
   yolo pose train model=yolo11s-pose.pt data=datasets/swd_pose_split_0.7_0.2_0.1/swd_pose.yaml imgsz=1280 epochs=150 batch=16 mosaic=0 fliplr=0.5 degrees=5 scale=0.2
✅ Upscale done.


In [11]:
if __name__ == "__main__":
    # 划分比例：合计≈1.0；最终以 test = N - train - val 兜底
    SPLITS = {"train": 0.4, "val": 0.3, "test": 0.3}
    DEST_DIR = Path(f"datasets/swd_pose_split_{SPLITS['train']}_{SPLITS['val']}_{SPLITS['test']}")
    main(SPLITS, DEST_DIR)
    # for split in ["train","val","test"]:
    #     SRC = DEST_DIR / "images"  # 原 images 根
    #     LAB = DEST_DIR / "labels"   # 原 labels 根
    #     DST = DEST_DIR / "images"  # 放大后的 images 根
    #     LAB_DST = DEST_DIR / "labels" # labels 同步复制
    #     process_split(split, SRC, LAB, DST, LAB_DST)

    print("✅ Upscale done.")

[INFO] 收集到样本对：887（跳过负样本：0）
[INFO] 划分结果： train=354, val=266, test=267
[OK] train images=354  labels=354
[OK] val   images=266  labels=266
[OK] test  images=267  labels=267
[OK] 写出 YAML: datasets/swd_pose_split_0.4_0.3_0.3/swd_pose.yaml
✅ Done. 数据已整理为 YOLO-Pose 标准结构。
   训练命令示例：
   yolo mode=checks data=datasets/swd_pose_split_0.4_0.3_0.3/swd_pose.yaml
   yolo pose train model=yolo11s-pose.pt data=datasets/swd_pose_split_0.4_0.3_0.3/swd_pose.yaml imgsz=1280 epochs=150 batch=16 mosaic=0 fliplr=0.5 degrees=5 scale=0.2
✅ Upscale done.


In [12]:
if __name__ == "__main__":
    # 划分比例：合计≈1.0；最终以 test = N - train - val 兜底
    SPLITS = {"train": 0.5, "val": 0.3, "test": 0.2}
    DEST_DIR = Path(f"datasets/swd_pose_split_{SPLITS['train']}_{SPLITS['val']}_{SPLITS['test']}")
    main(SPLITS, DEST_DIR)
    # for split in ["train","val","test"]:
    #     SRC = DEST_DIR / "images"  # 原 images 根
    #     LAB = DEST_DIR / "labels"   # 原 labels 根
    #     DST = DEST_DIR / "images"  # 放大后的 images 根
    #     LAB_DST = DEST_DIR / "labels" # labels 同步复制
    #     process_split(split, SRC, LAB, DST, LAB_DST)

    print("✅ Upscale done.")

[INFO] 收集到样本对：887（跳过负样本：0）
[INFO] 划分结果： train=443, val=266, test=178
[OK] train images=443  labels=443
[OK] val   images=266  labels=266
[OK] test  images=178  labels=178
[OK] 写出 YAML: datasets/swd_pose_split_0.5_0.3_0.2/swd_pose.yaml
✅ Done. 数据已整理为 YOLO-Pose 标准结构。
   训练命令示例：
   yolo mode=checks data=datasets/swd_pose_split_0.5_0.3_0.2/swd_pose.yaml
   yolo pose train model=yolo11s-pose.pt data=datasets/swd_pose_split_0.5_0.3_0.2/swd_pose.yaml imgsz=1280 epochs=150 batch=16 mosaic=0 fliplr=0.5 degrees=5 scale=0.2
✅ Upscale done.


In [13]:
if __name__ == "__main__":
    # 划分比例：合计≈1.0；最终以 test = N - train - val 兜底
    SPLITS = {"train": 0.5, "val": 0.2, "test": 0.3}
    DEST_DIR = Path(f"datasets/swd_pose_split_{SPLITS['train']}_{SPLITS['val']}_{SPLITS['test']}")
    main(SPLITS, DEST_DIR)
    # for split in ["train","val","test"]:
    #     SRC = DEST_DIR / "images"  # 原 images 根
    #     LAB = DEST_DIR / "labels"   # 原 labels 根
    #     DST = DEST_DIR / "images"  # 放大后的 images 根
    #     LAB_DST = DEST_DIR / "labels" # labels 同步复制
    #     process_split(split, SRC, LAB, DST, LAB_DST)

    print("✅ Upscale done.")

[INFO] 收集到样本对：887（跳过负样本：0）
[INFO] 划分结果： train=443, val=177, test=267
[OK] train images=443  labels=443
[OK] val   images=177  labels=177
[OK] test  images=267  labels=267
[OK] 写出 YAML: datasets/swd_pose_split_0.5_0.2_0.3/swd_pose.yaml
✅ Done. 数据已整理为 YOLO-Pose 标准结构。
   训练命令示例：
   yolo mode=checks data=datasets/swd_pose_split_0.5_0.2_0.3/swd_pose.yaml
   yolo pose train model=yolo11s-pose.pt data=datasets/swd_pose_split_0.5_0.2_0.3/swd_pose.yaml imgsz=1280 epochs=150 batch=16 mosaic=0 fliplr=0.5 degrees=5 scale=0.2
✅ Upscale done.


In [14]:
if __name__ == "__main__":
    # 划分比例：合计≈1.0；最终以 test = N - train - val 兜底
    SPLITS = {"train": 0.6, "val": 0.2, "test": 0.2}
    DEST_DIR = Path(f"datasets/swd_pose_split_{SPLITS['train']}_{SPLITS['val']}_{SPLITS['test']}")
    main(SPLITS, DEST_DIR)
    # for split in ["train","val","test"]:
    #     SRC = DEST_DIR / "images"  # 原 images 根
    #     LAB = DEST_DIR / "labels"   # 原 labels 根
    #     DST = DEST_DIR / "images"  # 放大后的 images 根
    #     LAB_DST = DEST_DIR / "labels" # labels 同步复制
    #     process_split(split, SRC, LAB, DST, LAB_DST)

    print("✅ Upscale done.")

[INFO] 收集到样本对：887（跳过负样本：0）
[INFO] 划分结果： train=532, val=177, test=178
[OK] train images=532  labels=532
[OK] val   images=177  labels=177
[OK] test  images=178  labels=178
[OK] 写出 YAML: datasets/swd_pose_split_0.6_0.2_0.2/swd_pose.yaml
✅ Done. 数据已整理为 YOLO-Pose 标准结构。
   训练命令示例：
   yolo mode=checks data=datasets/swd_pose_split_0.6_0.2_0.2/swd_pose.yaml
   yolo pose train model=yolo11s-pose.pt data=datasets/swd_pose_split_0.6_0.2_0.2/swd_pose.yaml imgsz=1280 epochs=150 batch=16 mosaic=0 fliplr=0.5 degrees=5 scale=0.2
✅ Upscale done.
